In [7]:
!pip install ibm_db

    100% |████████████████████████████████| 563kB 2.0MB/s 
  Running setup.py bdist_wheel for ibm-db ... - \ | / - \ | / - \ | / - \ done
  Stored in directory: /gpfs/fs01/user/s824-ebceb0a4aaced1-5b6b6a42040f/.cache/pip/wheels/d7/05/e2/d7b2f153bfbabcdf8af0fec36d78656142b5966cf6be091af3
Successfully built ibm-db


In [16]:
!pip install ibm_db_dbi

  Could not find a version that satisfies the requirement ibm_db_dbi (from versions: )
No matching distribution found for ibm_db_dbi


In [17]:
import ibm_db, ibm_db_dbi;

In [18]:
username = "pchatte";
password = "Gurbyb11"

In [19]:
database = "bigsql";
hostname = "iop-bi-master.imdemocloud.com";
port = "32051"

In [20]:
import ibm_db, ibm_db_dbi;
conn_string = (
      "DRIVER={{IBM DB2 ODBC DRIVER}};"
      "DATABASE={0};"
      "HOSTNAME={1};"
      "PORT={2};"
      "PROTOCOL=TCPIP;"
      "UID={3};"
      "PWD={4};").format(database, hostname, port, username, password);

conn = ibm_db.connect(conn_string, "", "")

In [21]:
query = "select count(*) from pchatte.offers";
stmt = ibm_db.exec_immediate(conn, query);
ibm_db.fetch_both(stmt)

{0: '94', '1': '94'}

In [24]:
import pandas as pd
import ibm_db_dbi
pconn = ibm_db_dbi.Connection(conn)
df_offers = pd.read_sql('SELECT * FROM pchatte.offers', pconn)
df_offers.columns = ["offer_id", "campaign", "varietal", "min_qty", "discount", "origin", "past_peak"]
df_offers.head()

,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,0,January,Malbec,72,56,France,FALSE
1,1,January,Pinot Noir,72,17,France,FALSE
2,2,February,Espumante,144,32,Oregon,TRUE
3,3,February,Champagne,72,48,France,TRUE
4,4,February,Cabernet Sauvignon,144,44,New Zealand,TRUE


In [26]:
df_transactions = pd.read_sql('SELECT * FROM pchatte.transactions', pconn)
df_transactions.columns = ["customer_name", "offer_id", "description"]
df_transactions['n'] = 1
df_transactions.head()

,customer_name,offer_id,description,n
0,Smith,2,coupon,1
1,Smith,24,coupon,1
2,Johnson,17,coupon,1
3,Johnson,24,coupon,1
4,Johnson,26,coupon,1


In order to segment our customers, we need a way to compare them. To do this we're going to use K-Means clustering. K-means is a way of taking a dataset and finding groups (or clusters) of points that have similar properties. K-means works by grouping the points together in such a way that the distance between all the points and the midpoint of the cluster they belong to is minimized.

In [27]:
# join the offers and transactions table
df = pd.merge(df_offers, df_transactions)




In [28]:
# create a "pivot table" which will give us the number of times each customer responded to a given offer
matrix = df.pivot_table(index=['customer_name'], columns=['offer_id'], values='n')
# a little tidying up. fill NA values with 0 and make the index into a column
matrix = matrix.fillna(0).reset_index()
# save a list of the 0/1 columns. we'll use these a bit later
x_cols = matrix.columns[1:]

Now to create the clusters, we're going to use the KMeans functionality from scikit-learn. I arbitrarily chose 5 clusters.

In [29]:
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=5)
# slice matrix so we only include the 0/1 indicator columns in the clustering
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[2:]])
matrix.cluster.value_counts()

0    39
3    22
1    18
4    15
2     3
Name: cluster, dtype: int64

Visualizing the Cluster:
Using PCA to convert from multi-dimensional to 2 dimensions

In [30]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
matrix['x'] = pca.fit_transform(matrix[x_cols])[:,0]
matrix['y'] = pca.fit_transform(matrix[x_cols])[:,1]
matrix = matrix.reset_index()

customer_clusters = matrix[['customer_name', 'cluster', 'x', 'y']]
customer_clusters.head()

offer_id,customer_name,cluster,x,y
0,Adams,3,-0.977592,0.136707
1,Allen,0,0.343786,0.010889
2,Anderson,4,0.438798,1.016915
3,Bailey,0,-0.257865,0.064824
4,Baker,0,-0.222225,-0.123091


In [31]:


df = pd.merge(df_transactions, customer_clusters)
df = pd.merge(df_offers, df)
df



,offer_id,campaign,varietal,min_qty,discount,origin,past_peak,customer_name,description,n,cluster,x,y
0,1,January,Pinot Noir,72,17,France,FALSE,Thomas,coupon,1,0,1.129729,0.004645
1,1,January,Pinot Noir,72,17,France,FALSE,Thomas,coupon,1,0,1.129729,0.004645
2,1,January,Pinot Noir,72,17,France,FALSE,Jackson,coupon,1,1,0.886591,-0.515432
3,1,January,Pinot Noir,72,17,France,FALSE,Jackson,coupon,1,1,0.886591,-0.515432
4,1,January,Pinot Noir,72,17,France,FALSE,Mitchell,coupon,1,0,0.384101,0.242153
5,1,January,Pinot Noir,72,17,France,FALSE,Mitchell,coupon,1,0,0.384101,0.242153
6,1,January,Pinot Noir,72,17,France,FALSE,Peterson,coupon,1,4,0.861221,0.360943
7,1,January,Pinot Noir,72,17,France,FALSE,Peterson,coupon,1,4,0.861221,0.360943
8,1,January,Pinot Noir,72,17,France,FALSE,Wood,coupon,1,0,0.447292,-0.077368
9,1,January,Pinot Noir,72,17,France,FALSE,Wood,coupon,1,0,0.447292,-0.077368


As a baseline, take a look at the varietal counts for cluster 1 vs. everyone else. It turns out that almost all of the Chardonnay offers were purchased by members of cluster 1. In addition, none of the Malbec offers were purchased by members of cluster 1.

In [32]:
df['is_1'] = df.cluster==1
df.groupby("is_1").varietal.value_counts()

is_1   varietal          
False  Champagne             428
       Chardonnay            224
       Cabernet Sauvignon    204
       Prosecco              196
       Espumante             160
       Pinot Noir            104
       Merlot                 96
       Malbec                 72
       Pinot Grigio           16
True   Chardonnay            176
       Champagne              92
       Prosecco               84
       Cabernet Sauvignon     64
       Pinot Noir             60
       Merlot                 48
       Pinot Grigio           12
       Espumante               8
dtype: int64

You can also segment out numerical features. For instance, look at how the mean of the min_qty field breaks out between 1 vs. non-1. It seems like members of cluster 1 like to by in bulk!

In [33]:
df.groupby("is_1")[['min_qty', 'discount']].mean()

,min_qty,discount
is_1,,
False,66.032000,59.786667
True,94.455882,51.316176
